In [1]:
import pandas as pd
import numpy as np
import datetime
import os

#記号は基本　規定値かndarray 行数は8760で処理

#有効数字2桁
%precision 2



#太陽光発電初期値

#インプットデータ
input_data = pd.read_csv('hakata_ekimae_2017_data_from_meteoblue.csv') #データ1
        #the global horizontal radiation on the earth's surface averaged over the time step [MJ/m2]
input_rad = pd.read_csv('rad.csv')#encordingはUTF-8なので注意 unitは、MJ/m2
input_csv_parameter = pd.read_csv('PV_input_parameter.csv')

#1列目以外を抽出
input_csv_parameter = input_csv_parameter.iloc[:, [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]]

Gsc = float(input_csv_parameter.iat[0,0])#float(1.367)#the solar constant [1.367 kW/m2]
Gt_NOCT = float(input_csv_parameter.iat[0,1]) #the solar radiation at which the NOCT is defined [0.8 kW/m2]
Ta_NOCT = float(input_csv_parameter.iat[0,2])#the ambient temperature at which the NOCT is defined [20°C]
Tc_NOCT = float(input_csv_parameter.iat[0,3]) #the nominal operating cell temperature [°C]
Tc_STC = float(input_csv_parameter.iat[0,4]) #the PV cell temperature under standard test conditions [25°C]

YPV = float(input_csv_parameter.iat[0,6]) #the rated capacity of the PV array, meaning its power output under standard test conditions [kW]
ap = float(input_csv_parameter.iat[0,7]) #Module Efficiency(Pmax)[%/℃]
beta= float(input_csv_parameter.iat[0,8])  #the slope of the surface [°]
eta_c = float(input_csv_parameter.iat[0,9]) #the solar absorptance and the solar transmittance[%]
fpv = float(input_csv_parameter.iat[0,10]) #the PV derating factor [%]
ganma = float(input_csv_parameter.iat[0,11]) #the azimuth of the surface [°]
lam=float(input_csv_parameter.iat[0,12])#the latitude [°]
phi=float(input_csv_parameter.iat[0,13])
rho_g = float(input_csv_parameter.iat[0,14]) # the ground reflectance, which is also called the albedo [%]
tau_a = float(input_csv_parameter.iat[0,15]) #the solar absorptance and the solar transmittance[%]
_Gt_STC =float(1) #the incident radiation at standard test conditions [1 kW/m2]
UL = tau_a*Gt_NOCT/(Tc_NOCT-Ta_NOCT)
total_cost = float(input_csv_parameter.iat[0,16]) #total cost
Op_year = float(input_csv_parameter.iat[0,17]) #oparation year
cost = total_cost/(Op_year*365*24)#cost[yen/kW/h]
#列名の変更
input_data.columns = ['Year', 'Month', 'Day','Hour', 'Minute','Temperature(2m)', 'Mean Sea Level Pressure[hpa]', \
                          'Shortwave Radiation[W/m2]','Wind Speed(10m)[m/s]','Wind Direction(10m)' ,'Wind Speed(80m)[m/s]'\
                  , 'Wind Direction(80m)']

#日付型などの変更
input_data['Date'] = input_data[['Year', 'Month', 'Day']].apply(lambda x: '{}@{}@{}'.format(x[0], x[1], x[2]), axis=1)
Date = input_data['Date']
input_data['Date'] =pd.to_datetime(Date, format='%Y@%m@%d')

input_data['Time'] = input_data[['Year', 'Month', 'Day','Hour']].apply(lambda x: '{}@{}@{}@{}'.format(x[0], x[1], x[2],x[3]), axis=1)
Time = input_data['Time']
input_data['Time'] =pd.to_datetime(Time, format='%Y@%m@%d@%H')

### nの作成

def date0():
    n  = []  # 結果を入れるリストを用意
    day = 1
    j = 0
    for num in range(24):
        if j == 24:
            day +=1
            j=0
        else:
            j +=1
        n.append(day)  # 結果を追加
    return n  # 結果のリストを返す

def date1():
    n  = []  # 結果を入れるリストを用意
    day = 2
    j = 0
    for num in range(len(input_data)-24):
        if j == 24:
            day +=1
            j=1
        else:
            j +=1
        n.append(day)  # 結果を追加
    return n  # 結果のリストを返す

n0 = date0()
n1 = date1()
n0
n0.extend(n1)#リストn0にn1を合体

input_data['n']  = pd.Series(np.array(n0))

### nの作成完了

    #列の順番変更
        #lst = input_data.columns.tolist()
        #lst.remove('Date')
        #lst.remove('Time')
        #lst.remove('n')
        #lst.insert(0, 'Time')
        #lst.insert(1, 'Date')
        #lst.insert(8, 'n') 

In [2]:
#ωの計算
input_data['ts'] = input_data['Hour']
input_data['ω'] = (input_data['ts'].values - 12 )*15
omega = input_data['ω'].values

#Bの計算
input_data['B'] = 360*((input_data['n'].values - 1) / 365)

#Eup
input_data['Eup'] = 3.82*(0.000075+0.001868*np.cos(input_data['B'].values)\
                             -0.032077*np.sin(input_data['B'].values))
#Ebottom
input_data['Ebottom'] = 3.82*(-0.014615*np.cos(2*input_data['B'].values)\
                             -0.04089*np.sin(2*input_data['B'].values))
#δの計算
input_data['δ the solar declination']=23.45*np.sin(360*(284+input_data['n'].values)/365)

delta=input_data['δ the solar declination'].values

#cosθの計算
input_data['cosθ'] = np.sin(delta)*np.sin(phi)*np.cos(beta)\
                    -np.sin(delta)*np.cos(phi)*np.sin(beta)*np.cos(ganma)\
                    +np.cos(delta)*np.cos(phi)*np.cos(beta)*np.cos(omega)\
                    +np.cos(delta)*np.sin(phi)*np.sin(beta)*np.cos(ganma)*np.cos(omega)\
                    +np.cos(delta)*np.sin(beta)*np.sin(ganma)*np.sin(omega)
                    
cos_theta = input_data['cosθ'].values
                
#θの計算
input_data['θ(radian)'] = np.arccos(input_data['cosθ'].values)

input_data['θ(degree)'] = np.rad2deg(input_data['θ(radian)'].values)

#cosθzの計算
input_data['cosθz'] =np.cos(phi)*np.cos(delta)*np.cos(omega)\
                        +np.sin(phi)*np.sin(delta)

cos_theta_z = input_data['cosθz'].values

input_data['θz(radian)'] = np.arccos(input_data['cosθz'].values)

input_data['θz(degree)'] = np.rad2deg(input_data['θz(radian)'].values)

#theta degree ndarray
theta = input_data['θ(degree)'].values

#theta degree ndarray
theta_z = input_data['θz(degree)'].values

In [3]:
#Gon(the extraterrestrial normal radiation)[kW/m2]の計算
input_data['Gon(the extraterrestrial normal radiation)[kW/m2]'] \
=Gsc*(1+0.033*np.cos(360*input_data['n'].values/365))

_Gon =input_data['Gon(the extraterrestrial normal radiation)[kW/m2]'].values

#Go(the extraterrestrial horizontal radiation [kW/m2])の計算
input_data['Go(the extraterrestrial horizontal radiation)[kW/m2]'] \
=_Gon*np.abs(cos_theta_z)

Go = input_data['Go(the extraterrestrial horizontal radiation)[kW/m2]'] .values

In [4]:
#_Go(the extraterrestrial horizontal radiation averaged over the time step )[kW/m2]の計算

def step_minus(x):  #再帰関数　引数[2]-引数[1] ndarrayで返す ndarrayを引数
    x = list(x)
    y = [0]
    y.extend(x) #引数2
    x.extend([0]) #引数1
    y = np.array(y)
    x = np.array(x)
    ans = y - x
    return ans[1:]


input_data['_Go(the extraterrestrial horizontal radiation averaged over the time step )[kW/m2]'] \
=(12/np.pi)*_Gon*np.abs(np.cos(phi)*np.cos(delta)*step_minus(np.sin(omega))\
+(np.pi/180)*step_minus(omega)*np.sin(phi)*np.sin(delta))

_Go = input_data['_Go(the extraterrestrial horizontal radiation averaged over the time step )[kW/m2]'] 

#_G または　input_radの処理
    #欠損値を0で置き換える
input_rad = input_rad.fillna(0)
input_rad = input_rad.drop(8760)

    #input_radをMJ2kW
input_data['the global horizontal radiation[kW/m2]']=\
input_rad['the global horizontal radiation(MJ/㎡)'].values/3.6

_G = input_data['the global horizontal radiation[kW/m2]'].values

In [5]:
#ktの計算
input_data['kt(the clearness index)'] =_G/Go
kt = input_data['kt(the clearness index)'].values

#_Gdの計算

def if_kt(x):
    
    ans = np.where(x < 0.22, 1-0.09*x, \
             np.where(x > 0.8, 0.165, 0.95511-0.1604*x+4.388*x**2-16.638*x**3+12.336*x**4))
    return ans

input_data['_Gd(the diffuse radiation)[kW/m2]']=_G*if_kt(kt)
_Gd = input_data['_Gd(the diffuse radiation)[kW/m2]'].values

#_Gb(the beam radiation)の計算
input_data['_Gb(the beam radiation)[kW/m2]']=_G-_Gd

_Gb = input_data['_Gb(the beam radiation)[kW/m2]'].values

In [6]:
#Rbの計算
input_data['Rb']=cos_theta/cos_theta_z

Rb = input_data['Rb'].values

#Afの計算
input_data['Af']=_Gb/_Go

Af = input_data['Af'].values

#fの計算
input_data['f']=np.sqrt(_Gb/_G)
    #欠損値を0で置き換える
input_data=input_data.fillna(0)
f = input_data['f'].values

In [7]:
#_Gtの計算
input_data['_Gt']=(_Gb+_Gd*Af)*Rb\
                +_Gd*(1-Af)*((1+np.cos(beta)/2))*(1+f*np.sin(beta/2)**3)\
                +_G*rho_g*((1-np.cos(beta))/2)
_Gt= input_data['_Gt'].values     

In [8]:
#Tc(the PV cell temperature [°C])の計算
Ta = input_data['Temperature(2m)'].values
input_data['Tc(the PV cell temperature)[°C]']\
= Ta+_Gt*(tau_a/UL)*(1-eta_c/tau_a)

Tc = input_data['Tc(the PV cell temperature)[°C]'].values

#Ppv(the output of the PV array)[kW]の計算
input_data['Ppv(the output of the PV array)[kW]']\
=YPV*fpv*(_Gt/_Gt_STC)*(1+ap*(Tc-Tc_STC))

Ppv = input_data['Ppv(the output of the PV array)[kW]'].values

In [9]:
#入力初期値をデータフレームにする。
input_parameter_name=['Gsc','Gt_NOCT','Ta_NOCT','Tc_NOCT','Tc_STC','UL',\
                      'YPV','ap','beta','eta_c','fpv','ganma','lam','phi',\
                      'rho_g','tau_a','total_cost','Op_year','cost']

input_parameter_list=[Gsc,Gt_NOCT,Ta_NOCT,Tc_NOCT,Tc_STC,UL,\
                      YPV,ap,beta,eta_c,fpv,ganma,lam,phi,\
                      rho_g,tau_a,total_cost,Op_year,cost]

def float2DataFrame(name,list):
    df0=pd.DataFrame({str(name[0]): [list[0]]})
    for i in range(len(list)-1):
        a = float(list[i+1])
        df=pd.DataFrame({str(name[i+1]): [a]})
        df0 =pd.concat([df0, df], axis=1)
    return df0

input_parameter=float2DataFrame(input_parameter_name,input_parameter_list)


In [10]:
#データの書き出し
input_data.to_csv('PV_result.csv',encoding="shift_jis")   
input_parameter.to_csv('PV_input_parameter_result.csv',encoding="shift_jis")  